# Lab 3 Primer 

[Slides](https://real-time-dsp.github.io/lab/lab3/primer.slides.html)

Digital filters have several benefits compared to their analog counterparts

* Design can be modified/tuned with software changes only
* Exact initial conditions can be enforced
* Scale to extremely high order
* Can be designed for perfectly linear phase

Digital filters are used in a plethora of applications. To name a few,

* Speech and audio
* Image, video, and computer vision
* Sonar, radar, and medical imaging
* Digital communication (wifi, cellular, etc)

## Review of LTI systems and filters

* [Mathematical definitions](https://realtimedsp.net/handouts/LTI_systems_filters.html#mathematical-definitions)
* [Impulse response and convolution](https://realtimedsp.net/handouts/LTI_systems_filters.html#impulse-response-and-convolution)
* [Frequency response](https://realtimedsp.net/handouts/LTI_systems_filters.html#frequency-response)
* [Magnitude and phase response](https://realtimedsp.net/handouts/LTI_systems_filters.html#magnitude-and-phase-response)
* [Frequency Selectivity](https://realtimedsp.net/handouts/LTI_systems_filters.html#frequency-selectivity)

## Finite impulse response (FIR) filters

For a causal discrete-time FIR filter, each output sample $y[n]$ is computed by taking the inner product of the input samples $x[n]$ with the filter coefficients $b[n]$.

$$ y[n] = \sum_{m=0}^{N}{x[n-m]b[m]} $$

This is often visualized as a 'tapped delay line'

![](../img/tapped_delay.svg)

<p style="text-align:center;">
<img src="../_images/tapped_delay.svg" width=900 height=900 class="center">
</p>

## FIR filter design

For typical frequency selective filters, the design goals are threefold

* Minimize distortion of the signal in the passband(s)
* Attenuate the stopband as much as possible
* Keep the order of the filter low

As an example, consider the design of a bandpass filter. Ideally, the passband would have a gain of 0 dB and the two stopbands would have a gain of $-\infty$ dB. However, we know that such an ideal filter would require infinite delay. In practice, we tolerate some amount of distortion in the passband and accept a small amount of leakage through the stopband. This leads to two important design tradeoffs:

* Better frequency selectivity vs lower delay
* Performance in the passband(s) vs performance in the stopband(s)

![](../img/bandpass.svg)

<p style="text-align:center;">
<img src="../_images/bandpass.svg" width=900 height=900 class="center">
</p>

The [Parks-McClellan algorithm](https://en.wikipedia.org/wiki/Parks–McClellan_filter_design_algorithm) (aka equiripple) produces a set of filter coefficients that is optimal in the sense that it minimizes the worst deviation between the actual response and the desired response. When using this method, we have control over several parameters:

* Filter order $N$
  * Lower order will provide shorter delay
  * Higher order will provide better frequency selectivity
* Relative weights $W$
  * Specifies how to weight each passband and stopband when computing the error that is minimized by the algorithm
* Location of transition regions (specified by $f_{\text{pass}_i}$ and $f_{\text{stop}_i}$)
  * Transition regions cause to much distortion to function as a passband but not enough attenuation to function as a stopband
  * Smaller transition region increases the amount of usable bandwidth
  * Larger transition region will provide better performance in the passband and stopband

Other FIR filter design methods include [least-squares](https://en.wikipedia.org/wiki/Filter_design#Methodology) and [window design methods](https://en.wikipedia.org/wiki/Finite_impulse_response#Window_design_method).

# Week 2

## Frame-based processing

For a real-time system, data is constantly flowing in from the ADC and out to the DAC. We can configure how many samples to collect between interrupt triggers from the ADC.

In the extreme case, we can trigger an interrupt each time a new sample is available and immediately compute one output sample of the filter. alternatively, we can wait until a frame of $N_F$ samples is collected before triggering an interrupt then compute $N_F$ output samples of the filter.

There are a number of advantages to a frame-based implementation:

* Less overhead
  * Each interrupt requires a fixed number of processor cycles $N_I$.
  * For sample-by-sample processing, this cost is incurred for each sample
  * For frame-based processing, this is reduced to $N_I / N_F$ per sample
* Computational advantages
  * Algorithms to filter $N_f$ samples can be more efficient than algorithms that operate sample-by-sample
  * Allows use of single instruction multiple data (SIMD) operations
* Memory advantages
  * In some systems, large blocks of memory can be moved more efficiently.
  * Caching may be more efficient with large frames